In [76]:
class bond:
    def __init__(self, coupone=None, YTM=None, maturity=None, par=None, price=None, term_per_year=2):
        self.coupone = coupone
        self.maturity = maturity
        self.par = par
        self.term_per_year = term_per_year

        # Ensure at least one of YTM or price is provided
        if YTM is None and price is None:
            raise ValueError("Either YTM or price must be provided.")

        # Calculate YTM if not provided
        self.YTM = YTM if YTM is not None else self.calculate_YTM()
        
        self.cf = self.coupone * self.par / self.term_per_year
        self.periodic_rate = self.YTM / self.term_per_year
        self.terms = self.maturity * self.term_per_year
        
        # Calculate price if not provided
        self.price = price if price is not None else self.calculate_price()

        # Ensure values are set before dependent attributes
        if self.coupone is None or self.par is None or self.maturity is None:
            raise ValueError("Coupon rate, maturity, and par value must be provided.")

        self.macaulay_duration = self.calculate_macaulay_duration()
        self.modified_duration = self.calculate_modified_duration()
        
    def set_YTM(self, YTM):
        """
        Set YTM and update other attributes.
        """
        self.YTM = YTM
        self.periodic_rate = self.YTM / self.term_per_year
        
    def calculate_YTM(self, coupone, maturity, par, price, term_per_year):
        """
        Calculates YTM of a bond.
        """
        # Define the function to solve for
        def ytm_eqn(ytm, coupone, maturity, par, price, term_per_year):
            return sum([coupone / (1 + ytm / term_per_year) ** i for i in range(1, maturity * term_per_year + 1)]) + \
                   par / (1 + ytm / term_per_year) ** (maturity * term_per_year) - price

        # Use a numerical solver to find the root of the equation
        from scipy.optimize import newton
        return newton(ytm_eqn, 0.05, args=(coupone, maturity, par, price, term_per_year))
    
    def calculate_price(self, periodic_rate=None, terms=None, cf=None, par=None):
        """
        Calculates bond price with default or new parameters.
        """
        # Use instance attributes if no new values are given
        periodic_rate = periodic_rate if periodic_rate is not None else self.periodic_rate
        terms = terms if terms is not None else self.terms
        cf = cf if cf is not None else self.cf
        par = par if par is not None else self.par

        return sum([cf / (1 + periodic_rate) ** i for i in range(1, terms + 1)]) + \
               par / (1 + periodic_rate) ** terms

    def calculate_dv01(self, bp=1):
        """
        Computes DV01 by shifting yield up/down by 'bp' basis points.
        """
        delta_yield = bp / 10000  # Convert basis points to decimal
        delta_periodic_rate = delta_yield / self.term_per_year
        new_price = self.calculate_price(periodic_rate=self.periodic_rate + delta_periodic_rate)
        return new_price - self.price
    
    def calculate_macaulay_duration(self):
        """
        Calculates Macaulay duration of a bond.
        """
        md =  sum([i * self.cf / (1 + self.periodic_rate)**i for i in range(1, self.terms + 1)]) + \
                self.terms * self.par/(1+self.periodic_rate)**self.terms
        md = md/self.price
        return md/self.term_per_year
    def calculate_modified_duration(self):
        """
        Calculates modified duration of a bond.
        """
        return self.macaulay_duration/(1+self.periodic_rate)

In [28]:
bond1 = bond(0.09, 0.09, 5, 100)
bond2 = bond(0.09, 0.09, 25, 100)
bond3 = bond(0.06, 0.09, 5, 100)
bond4 = bond(0.06, 0.09, 25, 100)
bond5 = bond(0, 0.09, 5, 100)
bond6 = bond(0, 0.09, 25, 100)
bond_portfolio = [bond1, bond2, bond3, bond4, bond5, bond6]
for y in range(6, 11):
    for x in bond_portfolio:
        x.set_YTM(y/100)
        print(f"YTM: {y}%\tcoupon: {x.coupone}\tMaturity:{x.maturity}  \tDV01: {x.calculate_dv01():.4f}\tPrice: {x.calculate_price():.4f}")
    print()

YTM: 6%	coupon: 0.09	Maturity:5  	DV01: 0.0459	Price: 112.7953
YTM: 6%	coupon: 0.09	Maturity:25  	DV01: 0.1653	Price: 138.5946
YTM: 6%	coupon: 0.06	Maturity:5  	DV01: 0.0427	Price: 100.0000
YTM: 6%	coupon: 0.06	Maturity:25  	DV01: 0.1286	Price: 100.0000
YTM: 6%	coupon: 0	Maturity:5  	DV01: 0.0361	Price: 74.4094
YTM: 6%	coupon: 0	Maturity:25  	DV01: 0.0554	Price: 22.8107

YTM: 7%	coupon: 0.09	Maturity:5  	DV01: 0.0437	Price: 108.3166
YTM: 7%	coupon: 0.09	Maturity:25  	DV01: 0.1384	Price: 123.4556
YTM: 7%	coupon: 0.06	Maturity:5  	DV01: 0.0405	Price: 95.8417
YTM: 7%	coupon: 0.06	Maturity:25  	DV01: 0.1067	Price: 88.2722
YTM: 7%	coupon: 0	Maturity:5  	DV01: 0.0342	Price: 70.8919
YTM: 7%	coupon: 0	Maturity:25  	DV01: 0.0432	Price: 17.9053

YTM: 8%	coupon: 0.09	Maturity:5  	DV01: 0.0416	Price: 104.0554
YTM: 8%	coupon: 0.09	Maturity:25  	DV01: 0.1166	Price: 110.7411
YTM: 8%	coupon: 0.06	Maturity:5  	DV01: 0.0385	Price: 91.8891
YTM: 8%	coupon: 0.06	Maturity:25  	DV01: 0.0890	Price: 78.5178
YT

In [41]:
bond1 = bond(0.09, 0.09, 5, 100)
print(bond1.price)

100.00000000000006


In [ ]:
ChangeInBP = [-300, -200, -100, -50, -10, -1, 1, 10, 50, 100, 200, 300]
bond1 = bond(0.09, 0.09, 5, 100)
bond2 = bond(0.09, 0.09, 25, 100)
bond3 = bond(0.06, 0.09, 5, 100)
bond4 = bond(0.06, 0.09, 25, 100)
bond5 = bond(0, 0.09, 5, 100)
bond6 = bond(0, 0.09, 25, 100)
bond_portfolio = [bond1, bond2, bond3, bond4, bond5, bond6]
for bp in ChangeInBP:
    for x in bond_portfolio:
        print(f"change in bp: {bp}\tcoupon: {x.coupone}\tMaturity:{x.maturity}  \tDV01: {x.calculate_dv01(bp = bp):.4f} \tpercentage change in price: {x.calculate_dv01(bp = bp)/x.price*100 :.2f}%")
    print()

change in bp: -300	coupon: 0.09	Maturity:5  	DV01: 12.7953 	percentage change in price: 12.80%
change in bp: -300	coupon: 0.09	Maturity:25  	DV01: 38.5946 	percentage change in price: 38.59%
change in bp: -300	coupon: 0.06	Maturity:5  	DV01: 11.8691 	percentage change in price: 13.47%
change in bp: -300	coupon: 0.06	Maturity:25  	DV01: 29.6430 	percentage change in price: 42.13%
change in bp: -300	coupon: 0	Maturity:5  	DV01: 10.0166 	percentage change in price: 15.56%
change in bp: -300	coupon: 0	Maturity:25  	DV01: 11.7397 	percentage change in price: 106.04%

change in bp: -200	coupon: 0.09	Maturity:5  	DV01: 8.3166 	percentage change in price: 8.32%
change in bp: -200	coupon: 0.09	Maturity:25  	DV01: 23.4556 	percentage change in price: 23.46%
change in bp: -200	coupon: 0.06	Maturity:5  	DV01: 7.7108 	percentage change in price: 8.75%
change in bp: -200	coupon: 0.06	Maturity:25  	DV01: 17.9152 	percentage change in price: 25.46%
change in bp: -200	coupon: 0	Maturity:5  	DV01: 6.499

In [66]:
bond1 = bond(0.09, 0.09, 5, 100)
bond2 = bond(0.09, 0.09, 25, 100)
bond3 = bond(0.06, 0.09, 5, 100)
bond4 = bond(0.06, 0.09, 25, 100)
bond5 = bond(0, 0.09, 5, 100)
bond6 = bond(0, 0.09, 25, 100)
bond_portfolio = [bond1, bond2, bond3, bond4, bond5, bond6]
for b in bond_portfolio:
    print(f'{b.maturity}-year  \t{b.coupone*100}%  \tInitial Price: {b.price:.4f}  \tnew price: {b.calculate_price(0.0901/2):.4f} DV01:{b.calculate_dv01():.4f}')

5-year  	9.0%  	Initial Price: 100.0000  	new price: 99.9604 DV01:-0.0396
25-year  	9.0%  	Initial Price: 100.0000  	new price: 99.9013 DV01:-0.0987
5-year  	6.0%  	Initial Price: 88.1309  	new price: 88.0943 DV01:-0.0366
25-year  	6.0%  	Initial Price: 70.3570  	new price: 70.2824 DV01:-0.0746
5-year  	0%  	Initial Price: 64.3928  	new price: 64.3620 DV01:-0.0308
25-year  	0%  	Initial Price: 11.0710  	new price: 11.0445 DV01:-0.0265


In [77]:
bond4p5 = bond(coupone=0.09, YTM=0.09, maturity=5, par=100)
print(bond4p5.macaulay_duration)
print(bond4p5.modified_duration)

4.134395247540063
3.956359088555084


In [ ]:
q2bondA = bond(0.08, 0.08, 2, 100, 100)
q2bondB = bond(0.09, 0.08, 2, 100, 100)

print(q2bondA.calculate_dv01())
print(q2bondB.calculate_dv01())

0.036298953871373385
0.03672668738152396
